# Update existing DSC

The [Quick Start](quick_start.html) example contains 3 DSC blocks, `simulate`, `estimate` and `mse`. Suppose we now have a new computational routine for location estimate via the *Winsorized* mean. The routine is implemented as `winsor.R`:

```r
  mean = psych::winsor.mean(x, trim = trim, na.rm = TRUE)
```

There are two ways to incorporate this new computational routine into DSC: either by adding it to the existing `estimate` block, or by adding a new block. For `winsor.R` there is an additional parameter `trim` which does not exist in `estimate`. We will show here two flavors to update, both involving minimal changes to the DSC file and result in the same outcome. In practice one should always try to adopt a flavor that is most logically clear and succinct.

## Add Steps to Existing Block
We change the `estimate` block to the following:

```yaml
  estimate:
      exec: mean.R, median.R, winsor.R
      params:
          x: $x
          exec[3]:
              trim: 0.1, 0.2
      return: mean

```

where we appended the new executable `winsor.R` to the `exec` entry, along with an executable specific parameter specification in the `params` section to assign two possible values to `trim` which essentially provides two versions of Winsorization for comparison.

To run the new DSC,

In [1]:
! dsc settings_add_method_v1.dsc -j8

INFO: Checking R library psych ...
INFO: DSC script exported to settings_add_method_v1.html
INFO: Constructing DSC from settings_add_method_v1.dsc ...
simulate:1+estimate:1+mse:1: 100% [========================] 3 1.0/s in 00:00:03
simulate:1+estimate:2+mse:1: 100% [========================] 3 1.0/s in 00:00:03
simulate:1+estimate:3+mse:1: 100% [========================] 3 0.7/s in 00:00:04
simulate:2+estimate:1+mse:1: 100% [========================] 3 1.1/s in 00:00:02
simulate:2+estimate:2+mse:1: 100% [========================] 3 1.1/s in 00:00:02
simulate:2+estimate:3+mse:1: 100% [========================] 3 0.7/s in 00:00:04
DSC: 100% [================================================] 6 0.3/s in 00:00:20
INFO: Building output database dsc_result.rds ...
INFO: DSC complete!
INFO: Elapsed time 21.980 seconds.


Notice that compared to using `-f` flag the computation time above is shorter. This is because after running the Quick Start tutorial DSC2 has cached steps in that DSC script along with their outcome; thus here only newly added steps are executed. 

## Add a New Block
Here we show an alternative flavor to update the existing DSC file: we keep the `estimate` block as is but add a new block `estimate_winsor`, and modify the **run** sequence in the DSC section to include this new block:

```yaml
  estimate:
      exec: mean.R, median.R
      params:
          x: $x
      return: mean

  estimate_winsor:
      exec: winsor.R
      params:
          x: $x
          trim: 0.1, 0.2
      return: mean

  ...
  DSC:
      run: simulate *
           (estimate, estimate_winsor) *
           mse

  ...
```

In the **run** sequence, the previous `estimate` and the new `estimate_winsor` blocks will be executed in parallel to each other, both taking input from `simulate` block and generating output for `mse` block.

To run this version,

In [3]:
! dsc settings_add_method_v2.dsc -j8 

INFO: DSC script exported to settings_add_method_v2.html
INFO: Constructing DSC from settings_add_method_v2.dsc ...
simulate:1+estimate:1+mse:1: 100% [========================] 3 1.0/s in 00:00:02
simulate:1+estimate:2+mse:1: 100% [========================] 3 1.0/s in 00:00:03
simulate:2+estimate:1+mse:1: 100% [========================] 3 1.0/s in 00:00:02
simulate:2+estimate:2+mse:1: 100% [========================] 3 1.0/s in 00:00:03
simulate:1+estimate_winsor:1+mse_1:1: 100% [===============] 3 0.6/s in 00:00:04
simulate:2+estimate_winsor:1+mse_1:1: 100% [===============] 3 0.7/s in 00:00:04
DSC: 100% [================================================] 6 0.3/s in 00:00:21
INFO: Building output database dsc_result.rds ...
INFO: DSC complete!
INFO: Elapsed time 22.946 seconds.


The result is identical to that from previous section.
